<a href="https://colab.research.google.com/github/gavbdheiver/Create-pandas-Dataframe-by-appending-one-row-at-a-time.ipynb/blob/main/Stepwise_Feature_Selection_for_Statsmodels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
#Basic imports
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

#Read King County, WA Housing Data csv
url = "https://raw.githubusercontent.com/garrettwilliams90/Housing_Analysis/main/data/kc_house_data.csv"
df = pd.read_csv(url)
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,Average,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,Average,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,Average,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,Very Good,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,Average,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [25]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [26]:
data_column_category=['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15']

In [27]:
from sklearn.preprocessing import LabelEncoder

#Creating the object instance

label_encoder = LabelEncoder()

for i in data_column_category:

    df[i] = label_encoder.fit_transform(df[i])

print("Label Encoded Data: ")

df.head()

Label Encoded Data: 


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,15326,30,346,2,2,106,2533,0,2,4,0,8,120,0,55,0,66,2397,222,109,2164
1,14454,114,2088,2,7,503,3576,2,0,4,0,8,392,209,51,45,55,4477,160,208,3470
2,13493,132,177,1,2,37,5517,0,0,4,0,7,38,0,33,70,16,4646,246,491,3804
3,6367,114,2370,3,10,313,2123,0,0,4,4,8,86,291,65,0,58,2491,86,116,1748
4,4952,124,1964,2,6,235,4207,0,0,4,0,9,252,0,87,0,37,3438,434,235,3357


In [30]:
#Split our DataFrame into X (input) and y (output)
y = df['price']
X = df.drop('price', axis = 1)

#Create an empty dictionary that will be used to store our results
function_dict = {'predictor': [], 'r-squared':[]}
#Iterate through every column in X
for col in X.columns:
    #Create a dataframe called selected_X with only the 1 column
    selected_X = X[[col]]
    #Fit a model for our target and our selected column 
    model = sm.OLS(y, sm.add_constant(selected_X)).fit()
    #Predict what our target would be for our model
    y_preds = model.predict(sm.add_constant(selected_X))
    #Add the column name to our dictionary
    function_dict['predictor'].append(col)
    #Calculate the r-squared value between the target and predicted target
    r2 = np.corrcoef(y, y_preds)[0, 1]**2
    #Add the r-squared value to our dictionary
    function_dict['r-squared'].append(r2)
    
#Once it's iterated through every column, turn our dictionary into a DataFrame and sort it
function_df = pd.DataFrame(function_dict).sort_values(by=['r-squared'], ascending = False)
#Display only the top 5 predictors
display(function_df.head())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,predictor,r-squared
4,sqft_living,0.451595
18,sqft_living15,0.365552
11,sqft_above,0.330660
3,bathrooms,0.269629
16,lat,0.218843


In [31]:
def next_possible_feature (X_npf, y_npf, current_features, ignore_features=[]):
    '''
    This function will loop through each column that isn't in your feature model and 
    calculate the r-squared value if it were the next feature added to your model. 
    It will display a dataframe with a sorted r-squared value.
    X_npf = X dataframe
    y_npf = y dataframe
    current_features = list of features that are already in your model
    ignore_features = list of unused features we want to skip over
    '''   
    #Create an empty dictionary that will be used to store our results
    function_dict = {'predictor': [], 'r-squared':[]}
    #Iterate through every column in X
    for col in X_npf.columns:
        #But only create a model if the feature isn't already selected or ignored
        if col not in (current_features+ignore_features):
            #Create a dataframe called function_X with our current features + 1
            selected_X = X_npf[current_features + [col]]
            #Fit a model for our target and our selected columns 
            model = sm.OLS(y_npf, sm.add_constant(selected_X)).fit()
            #Predict what  our target would be for our selected columns
            y_preds = model.predict(sm.add_constant(selected_X))
            #Add the column name to our dictionary
            function_dict['predictor'].append(col)
            #Calculate the r-squared value between the target and predicted target
            r2 = np.corrcoef(y_npf, y_preds)[0, 1]**2
            #Add the r-squared value to our dictionary
            function_dict['r-squared'].append(r2)
    #Once it's iterated through every column, turn our dict into a sorted DataFrame
    function_df = pd.DataFrame(function_dict).sort_values(by=['r-squared'],\
                                                          ascending = False)
    #Display only the top 5 predictors
    display(function_df.head())

In [32]:
#Start with the first predictor, and see all possibilities for our second.
selected_features = ['sqft_living']
features_to_ignore = []
#Since our function's ignore_features list is already empty, we don't need to 
#include our features_to_ignore list.
next_possible_feature (X_npf=X, y_npf=y, current_features=selected_features)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,predictor,r-squared
15,lat,0.642092
12,yr_built,0.478024
17,sqft_living15,0.471325
14,zipcode,0.465941
7,view,0.465770


In [35]:
#Start with the first predictor, and see all possibilities for our second.
selected_features = ['lat']
features_to_ignore = []
#Since our function's ignore_features list is already empty, we don't need to 
#include our features_to_ignore list.
next_possible_feature (X_npf=X, y_npf=y, current_features=selected_features)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,predictor,r-squared
4,sqft_living,0.642092
17,sqft_living15,0.561756
11,sqft_above,0.554093
3,bathrooms,0.477425
2,bedrooms,0.338438


In [36]:
#Define an empty dataframe to capture the VIF scores
vif_2 = pd.DataFrame()
#Create a Dataframe with only our selected features 
X_2 = X[['lat', 'sqft_living']]
#Label the scores with their related columns
vif_2["features"] = X_2.columns
#For each column,run a variance_inflaction_factor against all other columns
# to get a VIF Factor score
vif_2["VIF"] = [variance_inflation_factor(X_2.values, i) \
                for i in range(len(X_2.columns))]
vif_2

,features,VIF
0,lat,2.550228
1,sqft_living,2.550228


In [37]:

#Select our first 2 features.
selected_features = ['lat', 'sqft_living']
#ignore mulitcollinear features
features_to_ignore = ['lat', 'long']
next_possible_feature (X_npf=X, y_npf=y, current_features=selected_features)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,predictor,r-squared
16,sqft_living15,0.660789
7,view,0.655979
12,yr_built,0.649903
8,condition,0.648002
2,bedrooms,0.646361


In [38]:
#Define an empty dataframe to capture the VIF scores
vif_3 = pd.DataFrame()
#Create a Dataframe with only our selected features 
X_3 = X[['lat', 'sqft_living', 'sqft_living15']]
#Label the scores with their related columns
vif_3["features"] = X_3.columns
#For each column,run a variance_inflaction_factor against all other columns
# to get a VIF Factor score
vif_3["VIF"] = [variance_inflation_factor(X_3.values, i) \
                for i in range(len(X_3.columns))]
vif_3

,features,VIF
0,lat,2.802222
1,sqft_living,8.253346
2,sqft_living15,8.824453


In [40]:

#Create a Dataframe with only our selected features 
X_3 = X[['lat', 'sqft_living', 'sqft_living15']]
#Label the scores with their related columns
vif_3["features"] = X_3.columns
#For each column,run a variance_inflaction_factor against all other columns
# to get a VIF Factor score
vif_3["VIF"] = [variance_inflation_factor(X_3.values, i) \
                for i in range(len(X_3.columns))]
vif_3

,features,VIF
0,lat,2.802222
1,sqft_living,8.253346
2,sqft_living15,8.824453


In [42]:
#Select our final 3 features.
final_features = ['lat', 'sqft_living', 'sqft_living15']
final_X = X[final_features]
final_model = sm.OLS(y, sm.add_constant(final_X)).fit()
final_model.summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.661
Model:                            OLS   Adj. R-squared:                  0.661
Method:                 Least Squares   F-statistic:                 1.402e+04
Date:                Mon, 28 Feb 2022   Prob (F-statistic):               0.00
Time:                        00:08:11   Log-Likelihood:            -1.6767e+05
No. Observations:               21597   AIC:                         3.353e+05
Df Residuals:                   21593   BIC:                         3.354e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          -301.0525     11.413    -26.378      0.000    -323.423    -278.682
lat               0.3243      0.003    109.821      0.000       0.319       0.330
sqft_living       2.1884      0.028     79.398      0.000       2.134       2.242
sqft_living15     1.2676      0.037     34.499      0.000       1.196       1.340
==============================================================================
Omnibus:                      174.990   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              180.051
Skew:                           0.214   Prob(JB):                     7.99e-40
Kurtosis:                       3.128   Cond. No.                     9.49e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.49e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""